## CRAFT 
- Character Region Awareness for Text Detection

### Introduction
- Scene text detection has attracted much attention in the computer vision field because of its numerous applications, such as instant translation, image retrieval, scene parsing, geo-location, and blind-navigation.

- However, they may suffer in difficult cases, such as texts that are curved, deformed, or extremely long, which are hard to detect with a single bounding box. 
    - 구부러지거나 변형되거나 극단적으로 긴 것들은 파악하기 어렵다

- Character-level awareness has many advantages when handling challenging texts by linking the successive characters in a bottom-up manner. 
    - 문자 수준의 인식은 바텀업 방식으로 문자로 연결하여 텍스트 처리 시 큰 이점을 가짐

- we propose a novel text detector that localizes the individual character regions and links the detected characters to a text instance 
    - 개별 문자 영역을 지역화하고, 감지 된 문자를 텍스트 인스턴스에 연결하는 새로운 텍스트 감지기를 제안

- CRAFT (Character Region Awareness for Text Detection) is designed with a convolutional neural-network producing the character **region score** and **affinity score**
    
- The Region score is used to localize individual characters in the image 
    - 각각의 문자가 이미지에서 어느정도 위치인지
- The Affinity score is used to group each character into a single instance 
    - 해당 픽셀이 인접한 두 문자의 중심일 확률 - 이 점수를 기반으로 개별 문자가 하나의 단어로 그룹화될 것인지

![craft_model_architecture](img/craft_model_architecture.png)


## Methodology
- Our main objective is to precisely localize each individual character in natural images.
    - 각각의 이미지를 글자별로 분류하는 것이 주 목적
- Predict character regions and the affinity between characters.
- Since there is no public character-level dataset available, the model is trained in a weakly-supervised manner.
    - 문자 수준의 dataset이 없어서, 약한 지도학습으로 학습됨.

## Architecture
- Backbone : Convolutional network architecture based on VGG-16 with batch normalization
- Decoding Part : Skip-connections (which is similar to U-Net)

<hr>


## Ground Truth Label Generation (라벨링 과정)
- For each training image, we generate the ground truth label for the region score and the affinity score with character-level bounding boxes.
    - 각 훈련 이미지에 대해 ground truth를 생성(region score, affinity score)
- 1) Prepare a 2-dimensional isotropic Gaussian map
- 2) Compute perspective transform between the Gaussian map region and each character box
- 3) Warp Gaussian map to the box area

- Affinity Score 
    - 1) By drawing diagonal lines to connect opposite corners of each character box.
    - 2) We can generate two triangles which we will refer to as the upper and lower character triangles
    - 3) Each adjacent character box pair, an affinity box is generated by setting the centers of the upper and lower triangles as corners of the box

![craft](img/craft.png)

### Ground Truth Label 문제점 및 해결법
- 이러한 Ground Truth 를 매 이미지마다 만들기 힘듬 (데이터셋 부족)
- 기존에 공개되어 있는 대부분의 데이터 집합들은 Word-level로 라벨링 되어있음

- **데이터 부족에 대한 해결법 : 이를 해결하기 위해 Word-level Annotation이 포함된 데이터 집합을 Character-box로 생성하는 방법을 제안**
- 즉 단어로 되어있는 데이터셋을 문자데이터로 찢어서 활용하는 법 제안.
    - Weakly Supervised learning
    - Psuedo Ground Truth
    
<hr>


## Weakly-Supervised Learning
- Unlike synthetic datasets, real images in a dataset usually have word-level annotations
    - 합성 데이터 이미지와 달리, 실제 이미지에는 일반적으로 단어 수준으로 되어있음.
- we generate character boxes from each word-level annotation in a weakly-supervised manner
    - 우리는 weakly-supervised manner를 통해서 단어박스를 생성함 
- When a real image with word-level annotations is provided, the learned interim model predicts the character region score of the cropped word images to generate character-level bounding boxes
    - 단어 수준 문자가 포함 된 실제 이미지가 제공되면 학습 된 중간 모델이 잘라낸 단어 이미지의 문자 영역 점수를 예측하여 문자 수준 경계 상자를 생성

![craft2](img/craft2.png)

<hr>



## Word-Level annotation to Character-level annotation
- 쉬운말로 textbox to charater box

- 1) The word-level images are cropped from the original images
- 2) The model(interim model) trained up to date predicts the region score
- 3) The watershed algorithm is used to split the character regions
- 4) The coordinates of the character boxes are transformed back into the original image coordinates using the inverse transform from the cropping step. (unwarp 과정)
- 단어 단위 로부터 문자 단위를 도출한 후 각 Region score map과 Affinity Score map을 구함 (Pseudo-Ground Truth)
- 이 Ground Truth는 Interim model에 의해 예측된 것이므로 Pseudo-Ground Truth라고 부름.

![wordlevel_to_characterlevel](img/wordlevel_to_characterlevel.png)



## Pseudo Ground Truth
- Pseudo Ground Truth는 Interim model로 예측된 결과임.
- 따라서 이것을 온전히 믿으면 안되고 신뢰도를 통해 최적화 하여야 함.
- When the model is trained using weak-supervision, we are compelled to train with incomplete pseudo-GTs.
    - 위에서 구한 방법으로 GT들이 생성되고 이것으로 훈련함. (이것이 정답 라벨이라고 생각하고 훈련해야함
    
- If the model is trained with inaccurate region scores, the output might be blurred within character regions. 

- To prevent this we measure the quality of each pseudo-GTs generated by the model
    - 단 위에서 방법이 잘못될수도 있기 때문에 구한 GT에 대한 신뢰도 필요
- There is a very strong cue in the text annotation, which is the word length
    - 신뢰도를 평가하는 좋은 기준 중 하나는 단어길이(word-length)이다.

- 신뢰도평가 ex)
    - 원래는 6글자, Splitting Characters 했더니 5글자
    - L(w) = 6, L^c(w) = 5  ==> conf(w) = 6 - min(6, 6-5)/6 => 5/6

![confidence](img/confidence.png)

- If the confidence score is below 0.5 ~ simply dividing the word region R(w) by the number of characters L(w)
    - 만일 신뢰도가 0.5 이하면, 단어간의 길이가 일정하다고 가정하고, 단어영역을 글자 길이로 나누어서 다시 진행함.

- 최종 Generate Confidence Map
    - 픽셀 p에 대해 p가 원본 이미지에서 Word-Box 영역에 포함된다면 좌표와 대응되는 Confidence Map의 좌표값은 WordBox의 Confidence Score로 설정
    - 나머지 영역은 모두 1
    
![Generate_Confidence_Score](img/Generate_Confidence_Score.png)

## 목적함수 (Loss)
- Region score and Affinity score respectively
- 각 픽셀 p에 대한 (Region / Affinity) 기댓값과 정답간의 유클리드
- 각각의 Loss의 합이 총 Loss

![craft_loss](img/craft_loss.png)


<hr>

## Word Level QuadBox Inference
- Region Score와 Affinity Score를 통해 각 문자를 하나의 단어로 그룹화
- 

## 모델 구조 특징
- Fully Convolutional network architecture
    - Bilinear interpolation
    - Backwards convolution 두가지 방법 사용
- VGG-16을 기반, Batch normalization을 사용
- U-net처럼 UpConv를 통해 Low-level 특징을 집계하여 예측함

## 참고문헌
- https://medium.com/@msmapark2/character-region-awareness-for-text-detection-craft-paper-%EB%B6%84%EC%84%9D-da987b32609c

- https://tkdguq05.github.io/2020/03/29/craft/ // CRAFT 요약